In [11]:
!pip install torch==2.7.1+cu126 torchvision torchaudio --index-url https://download.pytorch.org/whl/cu126

# Cài bitsandbytes
!pip install -U bitsandbytes

# Cài transformers, trl, peft, accelerate, datasets
!pip install transformers
!pip install trl
!pip install peft
!pip install accelerate
!pip install datasets
!pip install safetensors

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


Looking in indexes: https://download.pytorch.org/whl/cu126


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


In [1]:
import torch, transformers, trl, peft

print("torch:", torch.__version__)
print("transformers:", transformers.__version__)
print("trl:", trl.__version__)
print("peft:", peft.__version__)
print("CUDA:", torch.cuda.is_available())


torch: 2.7.1+cu126
transformers: 4.54.0
trl: 0.19.1
peft: 0.15.2
CUDA: True


In [5]:
import os
import gc
import torch
from datasets import load_dataset
from transformers import (
    AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig,
    HfArgumentParser, TrainingArguments, pipeline, logging,
    EarlyStoppingCallback
)
from peft import LoraConfig, PeftModel
from trl import SFTTrainer, SFTConfig

In [15]:
import json

input_file = 'medquad_dataset.jsonl'
output_file = 'medquad_llama_chat_format.jsonl'

with open(input_file, 'r') as infile, open(output_file, 'w') as outfile:
    for line in infile:
        item = json.loads(line)
        question = item['question'].strip().replace('\n', ' ')
        answer = item['answer'].strip().replace('\n', ' ')
        llama_chat_format = f"<s>[INST] {question} [/INST] {answer} </s>"
        outfile.write(json.dumps({"text": llama_chat_format}) + '\n')


In [2]:
import json
import random

# Đọc dữ liệu gốc
with open('medquad_llama_chat_format.jsonl', 'r') as f:
    data = [json.loads(line) for line in f]

# Xáo trộn dữ liệu
random.seed(42)
random.shuffle(data)

# Chia theo tỉ lệ 80% train, 10% valid, 10% test
n_total = len(data)
n_train = int(0.9 * n_total)

train_data = data[:n_train]
valid_data = data[n_train:]

# Ghi ra file
with open('train.jsonl', 'w') as f:
    for item in train_data:
        f.write(json.dumps(item) + '\n')

with open('valid.jsonl', 'w') as f:
    for item in valid_data:
        f.write(json.dumps(item) + '\n')


In [2]:
# base_model_name = 'NousResearch/Llama-2-7b-chat-hf'
# finetune_model_name='my_model_finetune_llama2_7b'

# output_dir = './results'

# no_of_epochs = 1

# # No change params
# use_4bit, bnb_4bit_compute_dtype, bnb_4bit_quant_type, use_nested_quant = True, "float16", "nf4", True # To quantization
# lora_r, lora_alpha, lora_dropout = 32, 8, 0.1
# fp16, bf16 =  False, False
# per_device_train_batch_size, per_device_eval_batch_size = 1, 1
# gradient_accumulation_steps, gradient_checkpointing, max_grad_norm = 1, True, 0.3
# learning_rate, weight_decay, optim = 2e-4, 0.001, "paged_adamw_32bit"
# lr_scheduler_type, max_steps, warmup_ratio = "cosine", -1, 0.03
# group_by_length, save_steps, logging_steps = True, 0, 25
# max_seq_length, packing, device_map = None, False, {"": 0}


base_model_name = 'NousResearch/Llama-2-7b-chat-hf'
finetune_model_name='my_model_finetune_llama2_7b'

output_dir = './results_2'

no_of_epochs = 15


# No change params
use_4bit, bnb_4bit_compute_dtype, bnb_4bit_quant_type, use_nested_quant = True, "int4", "nf4", True # To quantization
lora_r, lora_alpha, lora_dropout = 32, 8, 0.1
fp16, bf16 =  False, False
per_device_train_batch_size, per_device_eval_batch_size = 1, 1
gradient_accumulation_steps, gradient_checkpointing, max_grad_norm = 1, True, 0.3
learning_rate, weight_decay, optim = 2e-4, 0.001, "paged_adamw_32bit"
lr_scheduler_type, max_steps, warmup_ratio = "cosine", -1, 0.03
group_by_length, save_steps, logging_steps = True, 0, 25
max_seq_length, packing, device_map = 512, False, "auto",





In [3]:
compute_dtype = getattr(torch, bnb_4bit_compute_dtype)

from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig
import torch

bnb_config = BitsAndBytesConfig(
    load_in_4bit=use_4bit,
    bnb_4bit_quant_type=bnb_4bit_quant_type,
    bnb_4bit_compute_dtype=compute_dtype,
    bnb_4bit_use_double_quant=use_nested_quant,
)

# Load base model
model = AutoModelForCausalLM.from_pretrained(
    base_model_name,
    quantization_config=bnb_config,
    device_map=device_map
)

model.config.use_cache = False
model.config.pretraining_tp = 1

# Load LLaMA tokenizer
tokenizer = AutoTokenizer.from_pretrained(base_model_name, trust_remote_code=True)
tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "right" # Fix weird overflow issue with fp16 training

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [6]:
# Cấu hình LoRA
peft_config = LoraConfig(
    r=lora_r,
    lora_alpha=lora_alpha,
    target_modules=["q_proj", "v_proj"],
    lora_dropout=lora_dropout,
    bias="none",
    task_type="CAUSAL_LM",
)



training_arguments = TrainingArguments(
    output_dir=output_dir,
    num_train_epochs=no_of_epochs,
    per_device_train_batch_size=per_device_train_batch_size,
    gradient_accumulation_steps=gradient_accumulation_steps,
    learning_rate=learning_rate,
    weight_decay=weight_decay,
    logging_steps=logging_steps,
    save_steps=save_steps,
    eval_strategy ="steps",
    save_strategy="steps",
    load_best_model_at_end=True,
    metric_for_best_model="eval_loss",  # Dựa vào eval loss để chọn model tốt
    greater_is_better=False,           # Vì loss càng thấp càng tốt
    eval_steps=50,
    save_total_limit=2,
    fp16=fp16,
    bf16=bf16,
    max_grad_norm=max_grad_norm,
    warmup_ratio=warmup_ratio,
    lr_scheduler_type=lr_scheduler_type,
    report_to="wandb",   # hoặc "wandb", "tensorboard"
    remove_unused_columns=False,
)



In [ ]:
# trainer = SFTTrainer(
#     model=model,
#     train_dataset=load_dataset('json', data_files='train.jsonl', split='train'),
#     peft_config=peft_config,
#     dataset_text_field="text",
#     max_seq_length=max_seq_length,
#     tokenizer=tokenizer,
#     args=training_arguments,
#     packing=packing,
# )

# trainer.train()

import wandb
from datasets import load_dataset

# formatting function nếu bạn cần xử lý đầu vào
def formatting_func(example):
    return example["question"]  # hoặc "prompt" tuỳ theo bạn

# load dataset
dataset = load_dataset("json", data_files="train_convert.jsonl", split="train")
eval_dataset = load_dataset("json", data_files="valid_convert.jsonl", split="train")

    
def tokenize(example):
    text = formatting_func(example)
    return tokenizer(text, truncation=True, padding=False)


# init trainer
trainer = SFTTrainer(
    model=model,
    train_dataset=dataset, # Truyền dataset gốc vào đây
    # processing_class=tokenizer, # Sử dụng processing_class thay cho tokenizer
    args=training_arguments,
    eval_dataset=eval_dataset, # Thêm eval_dataset để đánh giá mô hình
    formatting_func=formatting_func, # SFTTrainer sẽ sử dụng hàm này để định dạng text trước khi tokenize
    peft_config=peft_config,# Đảm bảo max_seq_length được định nghĩa và truyền vào
    # callbacks=[EarlyStoppingCallback(early_stopping_patience=20)]  # dừng nếu eval loss không cải thiện sau 20 lần eval

)

trainer.train()



/home/student/miniconda3/envs/healthcare/lib/python3.9/site-packages/peft/mapping_func.py:73: UserWarning: You are trying to modify a model with PEFT for a second time. If you want to reload the model with a different config, make sure to call `.unload()` before.
  warnings.warn(
/home/student/miniconda3/envs/healthcare/lib/python3.9/site-packages/peft/tuners/tuners_utils.py:167: UserWarning: Already found a `peft_config` attribute in the model. This will lead to having multiple adapters in the model. Make sure to know what you are doing!
  warnings.warn(
No label_names provided for model class `PeftModelForCausalLM`. Since `PeftModel` hides base models input arguments, if label_names is not given, label_names can't be set automatically within `Trainer`. Note that empty label_names list will be used instead.


Step,Training Loss,Validation Loss
50,4.537700,4.493413
100,4.002000,3.843264
150,2.717400,2.446653
200,1.232100,1.033439
250,0.841000,0.772913
300,0.694600,0.660571
350,0.466500,0.479089
400,0.412700,0.491303
450,0.427700,0.464232
500,0.420900,0.454282


wandb: WARNING The get_url method is deprecated and will be removed in a future release. Please use `run.url` instead.


TrainOutput(global_step=52665, training_loss=0.25994657836815555, metrics={'train_runtime': 22162.4928, 'train_samples_per_second': 2.376, 'train_steps_per_second': 2.376, 'total_flos': 8.838889008058368e+16, 'train_loss': 0.25994657836815555})

In [11]:
wandb.finish()

eval/loss,▁▁▂▂▁▁▁▂▂▃▂▂▃▄▃▄▅▅▅▅▅▆▅▅▆▆▆▇▇███████████
eval/mean_token_accuracy,▇█▁██████████████████████▇▇███▇████▇▇▇▇▇
eval/num_tokens,▁▁▁▁▁▁▂▂▂▂▂▃▃▃▃▃▄▄▄▄▄▄▅▅▅▆▆▆▆▆▆▆▆▇▇▇▇███
eval/runtime,▂▂▁▂▂▂▁▂▁▁▂▁▂▂▂▂▂▂▂▃▂▂▃▃▃▃▃▃▃▃▄▄▆▆▆▆▆▇██
eval/samples_per_second,██████▇████▇█▇▇▇▇▇▇▇▆▆▆▅▅▆▅▅▅▅▄▃▃▃▃▂▁▁▂▁
eval/steps_per_second,█▇█▇████▇█▇▇▇▇▇▆▆▆▆▇▅▅▅▆▅▅▄▄▄▄▄▃▃▂▃▂▂▂▁▁
train/epoch,▁▁▁▁▂▂▂▂▃▃▃▃▃▃▃▄▅▅▅▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇▇▇▇███
train/global_step,▁▁▁▂▂▁▁▁▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▅▅▆▆▆▆▆▆▆▆▆▇▇▇▇██
train/grad_norm,▅▁▃▂▄▇▂▃▂▅▃▃▂▃▁▃▃▂▃▂▃▅▃▂▁█▂▃▅▂▃▃▂▂▃▂▃▃▂▃
train/learning_rate,▄▅████████████▇▇▇▇▆▆▆▆▅▅▅▅▄▄▄▃▃▂▂▂▂▁▁▁▁▁
train/loss,█▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁


In [12]:
model.eval()

LlamaForCausalLM(
  (model): LlamaModel(
    (embed_tokens): Embedding(32000, 4096, padding_idx=0)
    (layers): ModuleList(
      (0-31): 32 x LlamaDecoderLayer(
        (self_attn): LlamaAttention(
          (q_proj): lora.Linear4bit(
            (base_layer): Linear4bit(in_features=4096, out_features=4096, bias=False)
            (lora_dropout): ModuleDict(
              (default): Dropout(p=0.1, inplace=False)
            )
            (lora_A): ModuleDict(
              (default): Linear(in_features=4096, out_features=32, bias=False)
            )
            (lora_B): ModuleDict(
              (default): Linear(in_features=32, out_features=4096, bias=False)
            )
            (lora_embedding_A): ParameterDict()
            (lora_embedding_B): ParameterDict()
            (lora_magnitude_vector): ModuleDict()
          )
          (k_proj): Linear4bit(in_features=4096, out_features=4096, bias=False)
          (v_proj): lora.Linear4bit(
            (base_layer): Linear4bit(in

In [13]:
trainer.save_model("./my_finetuned_model_15ep_final")
tokenizer.save_pretrained("./my_finetuned_model_15ep_final")


('./my_finetuned_model_15ep_final/tokenizer_config.json',
 './my_finetuned_model_15ep_final/special_tokens_map.json',
 './my_finetuned_model_15ep_final/tokenizer.model',
 './my_finetuned_model_15ep_final/added_tokens.json',
 './my_finetuned_model_15ep_final/tokenizer.json')

In [1]:
from transformers import pipeline

pipe = pipeline("text-generation", model="./my_finetuned_model_15ep_final", tokenizer="./my_finetuned_model_15ep_final")

output = pipe("What are the symptoms of Odontomicronychial dysplasia ?", max_new_tokens=512, do_sample=True, temperature=0.7)
print(output[0]['generated_text'])


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Device set to use cuda:0


What are the symptoms of Odontomicronychial dysplasia ?

Odontomicronychial dysplasia is a rare genodermatosis that affects the development of teeth, nails, and skin. It is characterized by small, malformed teeth, thickened and brittle nails, and skin lesions. The skin lesions are typically found on the hands and feet and may be accompanied by nail dystrophy. The condition is usually discovered in early childhood and can be associated with a higher risk of infection.

The symptoms of Odontomicronychial dysplasia may include:

* Small, malformed teeth
* Thickened and brittle nails
* Skin lesions on the hands and feet
* Nail dystrophy
* Increased risk of infection

What are the causes of Odontomicronychial dysplasia?

The exact cause of Odontomicronychial dysplasia is not known, but it is thought to be related to genetic mutations. The condition is inherited in an autosomal dominant pattern, which means that a single copy of the mutated gene is enough to cause the condition.

What is the

In [ ]:
from transformers import pipeline
import torch

pipe = pipeline(
    "text-generation",
    model="./my_finetuned_model",
    tokenizer="./my_finetuned_model",
    torch_dtype=torch.bfloat16,     # hoặc torch.float16 nếu GPU không hỗ trợ bfloat16
    device_map="auto",              # tự động chọn GPU
    trust_remote_code=True          # cần cho tokenizer LLaMA
)

prompt = "What are the symptoms of Odontomicronychial dysplasia ?"

output = pipe(
    prompt,
    max_new_tokens=512,
    do_sample=True,
    temperature=0.7,
    top_p=0.9
)

print(output[0]['generated_text'])


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
Device set to use cuda:0


What are the symptoms of Odontomicronychial dysplasia ?

Odontomicronychial dysplasia is a rare genetic disorder that affects the development of the teeth, nails, and skin. The symptoms of this disorder can vary in severity and may include:

1. Teeth:
	* Delayed eruption of teeth (hypodontia)
	* Abnormal tooth shape and size
	* Teeth may be small, misshapen, or have an abnormal number of roots
	* Teeth may be loose or malformed
2. Nails:
	* Abnormal nail shape and size
	* Nails may be thin, brittle, or have an abnormal curvature
	* Nail bed may be thicker than normal
3. Skin:
	* Abnormal skin pigmentation or discoloration
	* Skin may be thin and fragile
	* Skin may have an abnormal texture or feel
4. Skeletal abnormalities:
	* Abnormal bone development or growth
	* Joint problems or dislocations
	* Short stature
5. Other symptoms:
	* Increased risk of infections
	* Poor wound healing
	* Abnormal sweat gland function
	* Abnormal taste or smell
6. Intellectual disability:
	* Mild to mode

: 